# 전이학습 실습
- ResNet50V2 사용해서 cats_n_dogs_small.zip
- 두 개 모델 개발
    1. classification만 학습하는 모델
        - classification은 직접 만들어서 학습
    2. fine tuning
        - conv5_block3_2_conv 이하 레이어들을 추가 학습

In [1]:
# 이미지 다운로드
import gdown
url = 'https://drive.google.com/uc?id=1nBE3N2cXQGwD8JaD0JZ2LmFD-n3D5hVU'
fname = 'cats_and_dogs_small.zip'
gdown.download(url, fname, quiet=False)

Downloading...
From: https://drive.google.com/uc?id=1nBE3N2cXQGwD8JaD0JZ2LmFD-n3D5hVU
To: /content/cats_and_dogs_small.zip
90.8MB [00:00, 145MB/s]


'cats_and_dogs_small.zip'

In [2]:
!mkdir data
!unzip -q ./cats_and_dogs_small.zip -d data/cats_and_dogs_small

In [3]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import applications
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array

import numpy as np

np.random.seed(0)
tf.random.set_seed(0)

In [4]:
LEARNING_RATE = 0.001
N_EPOCHS = 20
N_BATCHS = 100

IMAGE_SIZE = 224

## ImageDataGenerator 를 생성하는 함수

In [5]:
def get_generator(preprocess_input):
    """
    train/validation/test 용 ImageDataGenerator 생성해서 반환하는 함수
    [파라미터]
        preprocess_input: 전처리 함수
    [반환값]
        tuple: (train iter, validation iter, test iter)
    """
    train_dir = "data/cats_and_dogs_small/train/"
    val_dir = 'data/cats_and_dogs_small/validation/'
    test_dir = 'data/cats_and_dogs_small/test'
    
    # train generator: image augmentation 적용
    train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input, 
                                       rotation_range=40, 
                                       brightness_range=[0.7, 1.3], 
                                       zoom_range=0.2, 
                                       horizontal_flip=True, 
                                       width_shift_range=0.2, 
                                       height_shift_range=0.2, 
                                       fill_mode='constant')
    val_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
    test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
    
    train_iter = train_datagen.flow_from_directory(train_dir, target_size=(IMAGE_SIZE, IMAGE_SIZE), batch_size=N_BATCHS, class_mode='binary')
    val_iter = val_datagen.flow_from_directory(val_dir, target_size=(IMAGE_SIZE, IMAGE_SIZE), batch_size=N_BATCHS, class_mode='binary')
    test_iter = test_datagen.flow_from_directory(test_dir, target_size=(IMAGE_SIZE, IMAGE_SIZE), batch_size=N_BATCHS, class_mode='binary')
    
    return train_iter, val_iter, test_iter

In [6]:
def predict_cat_dog(img_path, model, preprocess_input, mode=False):
    """
    이미지를 받아서 모델을 이용해 추론 후 결과를 반환하는 함수
    [파라미터]
        img_path: str - 분류(추론)할 이미지 경로
        model: 학습된 모델 객체
        preprocess_input: 이미지 전처리 함수
        model: conv_base(convolution)을 통해 feature를 추출할지 여부
    [반환값]
        tuple: (예측확률, 예측라벨 index, 예측라벨명)
    """
    class_names = ['Cat', 'Dog']
    img = load_img(img_path, target_size=(224,224))
    img_np = img_to_array(img)
    img_np = img_np[np.newaxis, ...]
    
    input_tensor = preprocess_input(img_np)
    
    if mode:
        conv_base = applications.VGG16(include_top=False, input_shape=IMAGE_SIZE+(3,))
        input_tensor = conv_base.predict(input_tensor) #Feature map 추출
    
    
    pred = model.predict(input_tensor)
    pred_class = np.where(pred<=0.5, 0, 1)
#     print(pred_class)
    pred_label = class_names[pred_class[0,0]]
    
    return pred, pred_class, pred_label

## 전이학습 예제

In [7]:
def create_model_1():
    conv_base = applications.ResNet50V2(include_top=False, input_shape=(IMAGE_SIZE,IMAGE_SIZE,3))
    # non trainable로 바꾸기 - 학습이 안 되도록
    conv_base.trainable = False

    model = keras.Sequential()
    model.add(conv_base)
    
    # 출력 레이어(classification)
    model.add(layers.GlobalAveragePooling2D())
    model.add(layers.Dense(1, activation='sigmoid'))

    return model

In [8]:
model_1 = create_model_1()
model_1.compile(optimizer=keras.optimizers.Adam(learning_rate=LEARNING_RATE), loss='binary_crossentropy', metrics=['accuracy'])
model_1.summary()

94683136/94668760 [==============================] - 1s 0us/step
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50v2 (Functional)      (None, 7, 7, 2048)        23564800  
_________________________________________________________________
global_average_pooling2d (Gl (None, 2048)              0         
_________________________________________________________________
dense (Dense)                (None, 1)                 2049      
Total params: 23,566,849
Trainable params: 2,049
Non-trainable params: 23,564,800
_________________________________________________________________


In [9]:
train_iter, val_iter, test_iter = get_generator(applications.resnet50.preprocess_input)

Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


In [10]:
# 학습
hist = model_1.fit(train_iter, epochs=10, steps_per_epoch=len(train_iter), validation_data=val_iter, validation_steps=len(val_iter))

Epoch 1/10
20/20 [==============================] - 72s 2s/step - loss: 38.2875 - accuracy: 0.4830 - val_loss: 10.9378 - val_accuracy: 0.5290
Epoch 2/10
20/20 [==============================] - 39s 2s/step - loss: 20.4600 - accuracy: 0.4975 - val_loss: 13.3635 - val_accuracy: 0.5050
Epoch 3/10
20/20 [==============================] - 39s 2s/step - loss: 14.0622 - accuracy: 0.4955 - val_loss: 10.9707 - val_accuracy: 0.5350
Epoch 4/10
20/20 [==============================] - 39s 2s/step - loss: 12.9141 - accuracy: 0.5050 - val_loss: 7.9397 - val_accuracy: 0.5780
Epoch 5/10
20/20 [==============================] - 39s 2s/step - loss: 12.5589 - accuracy: 0.5005 - val_loss: 8.7885 - val_accuracy: 0.5530
Epoch 6/10
20/20 [==============================] - 39s 2s/step - loss: 10.8830 - accuracy: 0.5155 - val_loss: 7.3487 - val_accuracy: 0.5750
Epoch 7/10
20/20 [==============================] - 39s 2s/step - loss: 11.3445 - accuracy: 0.5185 - val_loss: 9.2681 - val_accuracy: 0.5450
Epoch 8/10

## fine tuning

In [11]:
def create_model_2():
    conv_base = applications.ResNet50V2(include_top=False, input_shape=(IMAGE_SIZE, IMAGE_SIZE, 3))

    # conv_base에서 conv5_block3_2_conv 이전 레이어(top layer 2개)들은 trainable, 나머지는 non trainable로 설정
    is_trainable = False
    for layer in conv_base.layers:
        if layer.name == 'conv5_block3_2_conv':
            is_trainable = True
        layer.trainable = is_trainable
        
    model = keras.Sequential()
    model.add(conv_base)
    model.add(layers.GlobalAveragePooling2D())
    model.add(layers.Dense(1, activation='sigmoid'))

    return model

In [12]:
model2 = create_model_2()
model2.compile(optimizer=keras.optimizers.Adam(learning_rate=LEARNING_RATE), loss='binary_crossentropy', metrics=['accuracy'])
model2.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50v2 (Functional)      (None, 7, 7, 2048)        23564800  
_________________________________________________________________
global_average_pooling2d_1 ( (None, 2048)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 2049      
Total params: 23,566,849
Trainable params: 3,417,089
Non-trainable params: 20,149,760
_________________________________________________________________


In [13]:
hist = model2.fit(train_iter, epochs=10, steps_per_epoch=len(train_iter), validation_data=val_iter, validation_steps=len(val_iter))

Epoch 1/10
20/20 [==============================] - 44s 2s/step - loss: 0.6967 - accuracy: 0.5215 - val_loss: 0.8886 - val_accuracy: 0.5060
Epoch 2/10
20/20 [==============================] - 39s 2s/step - loss: 0.6918 - accuracy: 0.5400 - val_loss: 0.7087 - val_accuracy: 0.5270
Epoch 3/10
20/20 [==============================] - 40s 2s/step - loss: 0.6886 - accuracy: 0.5425 - val_loss: 0.6855 - val_accuracy: 0.5420
Epoch 4/10
20/20 [==============================] - 39s 2s/step - loss: 0.6884 - accuracy: 0.5485 - val_loss: 0.6852 - val_accuracy: 0.5190
Epoch 5/10
20/20 [==============================] - 39s 2s/step - loss: 0.6806 - accuracy: 0.5680 - val_loss: 0.6857 - val_accuracy: 0.5210
Epoch 6/10
20/20 [==============================] - 40s 2s/step - loss: 0.6782 - accuracy: 0.5675 - val_loss: 0.6867 - val_accuracy: 0.5140
Epoch 7/10
20/20 [==============================] - 40s 2s/step - loss: 0.6800 - accuracy: 0.5560 - val_loss: 0.6881 - val_accuracy: 0.5330
Epoch 8/10
20/20 [==